# Lab 2: Working with Anthropic API and Reasoning Models


In [ ]:
# Import necessary libraries
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [2]:
client = anthropic.Client()

# you can use OpenAI APIs to call Anthropic models but you need to set the model parameter
# to an Anthropic model name
message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system="You are a helpful assistant.",
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=100,
    temperature=0.9
)

display(Markdown(message.content[0].text) if message.content else "No response")

The capital of France is Paris.

#### Step 3: Customize the Prompt

In [ ]:
# this is an example of how to build up the prompt to send to the model to create a customer service response 

# the customer complaint 
complaint = "You have over charged me for this product. I demand a refund."

# augment the prompt with company policy
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [ ]:
# Define the role and personality of the AI agent
system_message = "You are a customer service agent."

# Build the prompt to pass to the model
prompt = f"""

We have a user complaint: "{complaint}"
Company policy is as follows: {company_policy}

Your job is to give a response to the user complaint based on the company policy.
Keep your tone professional and empathetic, but also firm about the policy.

Keep your responses short and to the point, no more than 50 words.

"""
# Call the model with the prompt
chat_message = client.messages.create(
    model="claude-3-5-haiku-latest",
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=200,
    temperature=0.
)

# Diplay the response
chat_message = chat_message.content[0].text
display(Markdown(chat_message)) 


I apologize for your concern. Could you please provide your order date and total purchase amount? We'll review your refund request according to our policy and work to resolve this promptly and fairly.

#### Step 4: Rate the response using a reasoning model

In [ ]:
# Now we will use a more advanced model to reason about the response. Typically you use a chat model for initial interactions because they are faster. For more sensitive
# interactions you can use a reasoning model to evaluate the response from the chat model.
system_message = "You are a senior customer service agent with advanced reasoning capabilities. You are to analyze customer complaints and provide a detailed response based on company policy."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint and company policy
- Professional tone and empathy

"""
# Sonnet is still a chat model but it has advanced reasoning capabilities. It can also do "thinking" where it can 
# use a scratchpad to reason through complex problems. This is enabled with the thinking parameter.
reasoning_message = client.messages.create(
    model="claude-3-7-sonnet-latest",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000       # Optional, max tokens to use for thinking
    },
    max_tokens=3000,                # The max tokens must always be greater than the thinking budget
    temperature=1,                  # For thinking models, we must provide a number of 1.
    system=system_message,
    messages=[
        {"role": "user", "content": reasoning_prompt}
    ]
)

# display the raw response
reasoning_message = reasoning_message.content
print(reasoning_message)



[ThinkingBlock(signature='ErUBCkYICBgCIkCCjUB9d0qhmdSRxBlSpDJHu3rw+jciX/IMh5fufuV8zCpKtAjB2n+ejICX1+2r5rChG+qMKHi2t2DEvSxAqNYnEgxSUgMcmstunupV3e0aDK7uEuJCxOcUiY0LIiIwfNYtD8UrZQL1IVdNLhdFID5Oy9xxT7RrDyEHVy8nAkye0zuhszW3yJbdznuoWcTCKh2QGSt1cdeapHmAWMnbrGkB/xjs555oFpKPu9M5kBgC', thinking='Let me analyze this customer service situation and the provided response according to the requested criteria.\n\nFirst, let\'s look at the customer complaint:\n"You have over charged me for this product. I demand a refund."\n\nThe complaint is brief but indicates:\n1. The customer believes they\'ve been overcharged\n2. They\'re explicitly demanding a refund\n3. The tone suggests frustration\n\nNext, let\'s examine the response provided:\n"I apologize for your concern. Could you please provide your order date and total purchase amount? We\'ll review your refund request according to our policy and work to resolve this promptly and fairly."\n\nAnd finally, let\'s consider the company policy:\n- Refunds only

In [ ]:
# Now format the reponse. Note that the response has two parts, the thinking part and the final text part.
reasoning_thinking = reasoning_message[0].thinking
reasoning_message_text = reasoning_message[1].text

display(Markdown(reasoning_thinking))
display(Markdown( reasoning_message_text))

Let me analyze this customer service situation and the provided response according to the requested criteria.

First, let's look at the customer complaint:
"You have over charged me for this product. I demand a refund."

The complaint is brief but indicates:
1. The customer believes they've been overcharged
2. They're explicitly demanding a refund
3. The tone suggests frustration

Next, let's examine the response provided:
"I apologize for your concern. Could you please provide your order date and total purchase amount? We'll review your refund request according to our policy and work to resolve this promptly and fairly."

And finally, let's consider the company policy:
- Refunds only within 14 days of purchase
- Purchases over $100 require supervisor approval
- Balance between customer satisfaction and preventing refund abuse

Analysis of the response:

1. Appropriateness given complaint and policy:
   - The response acknowledges the concern with an apology
   - It appropriately asks for more information (order date and purchase amount) which is necessary to:
     a) Verify if the purchase falls within the 14-day refund window
     b) Determine if the amount exceeds $100 (which would require supervisor approval)
   - The response doesn't make promises that might contradict policy
   - It sets expectations that the refund request will be reviewed according to policy
   - It doesn't immediately deny or approve the refund without proper information

2. Professional tone and empathy:
   - The tone is professional and courteous
   - It begins with an apology, showing some empathy for the customer's concern
   - The language is clear and straightforward
   - It indicates a willingness to work toward resolution
   - It avoids defensive language despite the customer's demanding tone
   - It offers a balanced approach of fairness (suggesting both customer and company interests will be considered)

Overall evaluation:
The response is generally appropriate and professionally written. It correctly seeks the information needed to apply company policy without making promises that might violate those policies. It maintains a respectful tone despite the customer's frustration.

Potential improvements could include:
- A bit more acknowledgment of the specific concern about overcharging (not just a generic apology)
- Perhaps offering to explain how pricing works if there was a misunderstanding
- Providing a timeframe for when they can expect a response after providing the requested information

# Analysis of Customer Service Response

## Appropriateness Given Complaint and Policy

The response is appropriate and well-balanced for several reasons:

- **Information Gathering**: The agent correctly asks for the order date and purchase amount, which are essential to determine:
  - If the purchase falls within the 14-day refund window
  - Whether the amount exceeds $100 (requiring supervisor approval)

- **Policy Alignment**: The response mentions reviewing the request "according to our policy" without making promises that could contradict established guidelines.

- **Non-Committal**: The agent avoids immediately agreeing to the refund before verifying the claim of overcharging, which helps prevent potential refund abuse.

## Professional Tone and Empathy

The response demonstrates professionalism and appropriate empathy:

- **Acknowledgment**: Opens with an apology, showing respect for the customer's concern
- **Solution-oriented**: Focuses on gathering information to resolve the issue
- **Balanced language**: Uses terms like "promptly and fairly" to convey both customer service and adherence to policy
- **De-escalation**: Responds calmly to the demanding tone of the customer

## Suggestions for Improvement

While the response is generally effective, it could be enhanced by:

- Acknowledging the specific claim of overcharging more directly
- Providing a timeframe for when the customer can expect a resolution after submitting the requested information
- Offering contact options if the customer needs immediate assistance

Overall, this is an appropriate initial response that balances customer service with proper procedure.